# Description du projet : 

Contexte :  L'INSEE est l'institut officiel français qui collecte des données de tous types sur le territoire français. Elles peuvent être démographiques (Naissances, Décès, Densité de la population...), économiques (Salaires, Entreprises par activité / taille...) et plus encore.
Ces données peuvent être d'une grande aide pour observer et mesurer les inégalités au sein de la population française.

## Objectif : Comparer les inégalités en France : 

* Entreprises en fonction de leur localisation, de leur taille. 
* Population en fonction du salaire et de la localisation.
* Focus sur une grande ville 

## Ressources à consulter : 
* Données : https://assets-datascientest.s3.eu-west-1.amazonaws.com/notebooks/power_bi/power_bi_datasets_projet/french_industry.zip


### Informations sur le nombre d'entreprises dans chaque ville française classées par taille.
<u>base_etablissement_par_tranche_effectif.csv :</u>

* CODGEO : ID géographique de la ville
* LIBGEO : nom de la ville
* REG : numéro de région
* DEP : numéro de département
* E14TST : nombre total d'entreprises dans la ville
* E14TS0ND : nombre d'entreprises de taille inconnue ou nulle dans la ville
* E14TS1 : nombre d'entreprises de 1 à 5 employés dans la ville
* E14TS6 : nombre d'entreprises de 6 à 9 employés dans la ville
* E14TS10 : nombre d'entreprises de 10 à 19 employés dans la ville
* E14TS20 : nombre d'entreprises de 20 à 49 employés dans la ville
* E14TS50 : nombre d'entreprises de 50 à 99 employés dans la ville
* E14TS100 :  nombre d'entreprises de 100 à 199 employés dans la ville
* E14TS200 : nombre d'entreprises de 200 à 499 employés dans la ville
* E14TS500 : nombre d'entreprises de plus de 500 employés dans la ville

In [1]:
# Suppression des warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
# Lecture fichier csv
df1 = pd.read_csv('../0000-Initial/base_etablissement_par_tranche_effectif.csv',dtype={'REG':'str'})
# Tous les nom de ville mis à la première lettre en masjuscule
df1.LIBGEO = df1.LIBGEO.str.title()
# Suppression des accents les plus communs
df1['LIBGEO'] = df1['LIBGEO'].str.replace("É","E")
df1['LIBGEO'] = df1['LIBGEO'].str.replace("Î","I")
#correction code geo erroné commune "L'Oudon"
# Extraction des données GEO de la DF
df1_base = df1.iloc[:,0:4]
# Affichage des infos du dataset
df1_base.info()
df1_base.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36681 entries, 0 to 36680
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CODGEO  36681 non-null  object
 1   LIBGEO  36681 non-null  object
 2   REG     36681 non-null  object
 3   DEP     36681 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


,CODGEO,LIBGEO,REG,DEP
0,01001,L'Abergement-Clémenciat,82,01
1,01002,L'Abergement-De-Varey,82,01
2,01004,Ambérieu-En-Bugey,82,01
3,01005,Ambérieux-En-Dombes,82,01
4,01006,Ambléon,82,01


### Données géographiques sur les villes françaises (principalement la latitude et la longitude, mais aussi les codes et les noms des régions/départements).

<u>geo_2014_v2.csv : </u> Fichier de synthèse de Dominique avec les coordonnées GPS

* COM (équivalent CODGEO)
* Code_postal
* latitude
* longitude

<u>codes_geo_2014_v3.xlsx : </u> Synthèse des données issues de l'INSEE

* CODGEO
* EU_circo 
* REG
* DEP 
* LIBGEO
* nom_région 
* nom_département 
* chef.lieu_région


In [3]:
# Utilisation du fichier données GPS (étude Dominique)
geo = pd.read_csv('../2014/geo_2014_v2.csv',usecols=['COM','Code_postal','latitude','longitude'])
# Suppression des doublons
geo = geo.drop_duplicates(subset='COM',keep='first')
# Affichage des premières lignes
geo.head()

,COM,Code_postal,latitude,longitude
0,01001,1400.0,46.153721,4.925850
1,01002,1640.0,46.009606,5.428088
2,01004,1500.0,45.961049,5.372275
3,01005,1330.0,45.996164,4.911967
4,01006,1300.0,45.749886,5.594585


In [4]:
# Lecture du fichier csv - insee source année 2014
# Lecture avec nom des colonnes personnalisé
# df2_2014 = pd.read_excel('../2014/codes_geo_2014_v2.xlsx',dtype='str',header=0,names=['CODGEO','EU_circo',
#                                                                                       'code_région','numéro_département',
#                                                                                       'nom_commune','nom_région',
#                                                                                       'nom_département','chef.lieu_region'])
df2_2014 = pd.read_excel('../2014/codes_geo_2014_v3.xlsx',dtype='str')
# Affichage des infos
df2_2014.info() 
# Affichage des premières lignes
df2_2014.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36681 entries, 0 to 36680
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   CODGEO            36681 non-null  object
 1   EU_circo          36681 non-null  object
 2   REG               36681 non-null  object
 3   DEP               36681 non-null  object
 4   LIBGEO            36681 non-null  object
 5   nom_région        36681 non-null  object
 6   nom_département   36681 non-null  object
 7   chef.lieu_région  36681 non-null  object
dtypes: object(8)
memory usage: 2.2+ MB


,CODGEO,EU_circo,REG,DEP,LIBGEO,nom_région,nom_département,chef.lieu_région
0,01001,Sud-Est,82,01,Abergement-Clémenciat,Rhône-Alpes,Ain,Bourg-en-Bresse
1,01002,Sud-Est,82,01,Abergement-de-Varey,Rhône-Alpes,Ain,Bourg-en-Bresse
2,01004,Sud-Est,82,01,Ambérieu-en-Bugey,Rhône-Alpes,Ain,Bourg-en-Bresse
3,01005,Sud-Est,82,01,Ambérieux-en-Dombes,Rhône-Alpes,Ain,Bourg-en-Bresse
4,01006,Sud-Est,82,01,Ambléon,Rhône-Alpes,Ain,Bourg-en-Bresse


In [5]:
# Fusion des données geo et GPS
df2 = pd.merge(df2_2014,geo,how='left',left_on='CODGEO',right_on='COM')

In [6]:
# Suppression des doublons
df2 = df2.drop_duplicates(subset='CODGEO',keep='first')
# Vérification de la suppression
df2.loc[df2.duplicated(subset='CODGEO')]

,CODGEO,EU_circo,REG,DEP,LIBGEO,nom_région,nom_département,chef.lieu_région,COM,Code_postal,latitude,longitude


In [7]:
# Suppression des colonnes non utiles pour le prochain merge
df2_base = df2.drop(labels=['REG','DEP','LIBGEO','COM'],axis='columns')

In [8]:
# Vérification des doublons
df2_base.loc[df2_base.duplicated()]

,CODGEO,EU_circo,nom_région,nom_département,chef.lieu_région,Code_postal,latitude,longitude


In [9]:
# Verification des nan
df2_base.isna().sum()

CODGEO                 0
EU_circo               0
nom_région             0
nom_département        0
chef.lieu_région       0
Code_postal         1739
latitude            1739
longitude           1739
dtype: int64

In [10]:
# Fusion des DataFrame "Etablissement par tranche d'effectifs" et "données géographiques"
all_content = pd.merge(df1_base,df2_base,how='left',on='CODGEO')
all_content.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36681 entries, 0 to 36680
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CODGEO            36681 non-null  object 
 1   LIBGEO            36681 non-null  object 
 2   REG               36681 non-null  object 
 3   DEP               36681 non-null  object 
 4   EU_circo          36681 non-null  object 
 5   nom_région        36681 non-null  object 
 6   nom_département   36681 non-null  object 
 7   chef.lieu_région  36681 non-null  object 
 8   Code_postal       34942 non-null  float64
 9   latitude          34942 non-null  float64
 10  longitude         34942 non-null  float64
dtypes: float64(3), object(8)
memory usage: 3.4+ MB


In [11]:
# Taille de la DataFrame
all_content.isna().sum()

CODGEO                 0
LIBGEO                 0
REG                    0
DEP                    0
EU_circo               0
nom_région             0
nom_département        0
chef.lieu_région       0
Code_postal         1739
latitude            1739
longitude           1739
dtype: int64

In [12]:
# Recherche des duplications
all_content_dupli = all_content.loc[all_content.duplicated(subset=['CODGEO','LIBGEO'])]
# Affichage des duplications
all_content_dupli

,CODGEO,LIBGEO,REG,DEP,EU_circo,nom_région,nom_département,chef.lieu_région,Code_postal,latitude,longitude


In [13]:
# Affichage de la dimension de la DF all_content
all_content.shape

(36681, 11)

### Informations démographiques par ville, âge, sexe et mode de vie

<u>communes_population.xlsx : </u> Données INSEE du recensement de 2012

* CODGEO 
* Population_totale 

In [14]:
# Lecture du fichier de données population
population = pd.read_excel('communes_population-2012_2014.xlsx',usecols=['CODGEO','Population_totale'])
# Affichage des infos de la DF
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36683 entries, 0 to 36682
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CODGEO             36683 non-null  object
 1   Population_totale  36683 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 573.3+ KB


In [15]:
# On groupe les données par commune pour obtenir la population
# Création d'une nouvelle DF population
# population = pd.DataFrame(df4.groupby(['NIVGEO','CODGEO','LIBGEO'])['NB'].sum(),index=None)
# Affichage de la dimension
population.shape

(36683, 2)

In [16]:
# Verification des nan
population.isna().sum()

CODGEO               0
Population_totale    0
dtype: int64

In [17]:
# Fusion des DF salaires et population
all_content_pop = pd.merge(all_content,population,how='left',on=['CODGEO'])
# Affichage de la dimension de la DF
all_content_pop.shape

(36681, 12)

In [18]:
# Vérification des nan
all_content_pop.isna().sum()

CODGEO                  0
LIBGEO                  0
REG                     0
DEP                     0
EU_circo                0
nom_région              0
nom_département         0
chef.lieu_région        0
Code_postal          1739
latitude             1739
longitude            1739
Population_totale       0
dtype: int64

In [19]:
all_content_pop.loc[all_content_pop.Population_totale.isna()]

,CODGEO,LIBGEO,REG,DEP,EU_circo,nom_région,nom_département,chef.lieu_région,Code_postal,latitude,longitude,Population_totale


In [20]:
all_content_pop.CODGEO.duplicated().sum()

0

In [21]:
# CODGEO : ID géographique de la ville
# LIBGEO : nom de la ville
# REG : numéro de région
# DEP : numéro de département
# E14TST : nombre total d'entreprises dans la ville
# E14TS0ND : nombre d'entreprises de taille inconnue ou nulle dans la ville
# E14TS1 : nombre d'entreprises de 1 à 5 employés dans la ville
# E14TS6 : nombre d'entreprises de 6 à 9 employés dans la ville
# E14TS10 : nombre d'entreprises de 10 à 19 employés dans la ville
# E14TS20 : nombre d'entreprises de 20 à 49 employés dans la ville
# E14TS50 : nombre d'entreprises de 50 à 99 employés dans la ville
# E14TS100 : nombre d'entreprises de 100 à 199 employés dans la ville
# E14TS200 : nombre d'entreprises de 200 à 499 employés dans la ville
# E14TS500 : nombre d'entreprises de plus de 500 employés dans la ville
df1_E14T = df1.drop(labels=['LIBGEO','REG','DEP'],axis='columns')
df1_E14T

,CODGEO,E14TST,E14TS0ND,E14TS1,E14TS6,E14TS10,E14TS20,E14TS50,E14TS100,E14TS200,E14TS500
0,01001,25,22,1,2,0,0,0,0,0,0
1,01002,10,9,1,0,0,0,0,0,0,0
2,01004,996,577,272,63,46,24,9,3,2,0
3,01005,99,73,20,3,1,2,0,0,0,0
4,01006,4,4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
36676,97613,169,169,0,0,0,0,0,0,0,0
36677,97614,230,230,0,0,0,0,0,0,0,0
36678,97615,569,569,0,0,0,0,0,0,0,0
36679,97616,345,345,0,0,0,0,0,0,0,0


In [22]:
# Fusion de all content pop avec les données des entreprises 
all_content_pop_entr = pd.merge(all_content_pop,df1_E14T,how='right',on='CODGEO')
all_content_pop_entr.shape

(36681, 22)

In [23]:
all_content_pop_entr.isna().sum()

CODGEO                  0
LIBGEO                  0
REG                     0
DEP                     0
EU_circo                0
nom_région              0
nom_département         0
chef.lieu_région        0
Code_postal          1739
latitude             1739
longitude            1739
Population_totale       0
E14TST                  0
E14TS0ND                0
E14TS1                  0
E14TS6                  0
E14TS10                 0
E14TS20                 0
E14TS50                 0
E14TS100                0
E14TS200                0
E14TS500                0
dtype: int64

In [24]:
all_content_pop_entr.loc[all_content_pop_entr.E14TST.isna()]

,CODGEO,LIBGEO,REG,DEP,EU_circo,nom_région,nom_département,chef.lieu_région,Code_postal,latitude,...,E14TST,E14TS0ND,E14TS1,E14TS6,E14TS10,E14TS20,E14TS50,E14TS100,E14TS200,E14TS500


### Salaires par villes française par catégories d'emploi, âge et sexe

<u>net_salary_per_town_categories.csv :</u> 

* CODGEO : ID géographique de la ville
* LIBGEO : nom de la ville
* SNHM14 : salaire net moyen
* SNHMC14 : salaire net moyen par heure pour les cadres
* SNHMP14 : salaire net moyen par heure pour un cadre moyen
* SNHME14 : salaire net moyen par heure pour l'employé
* SNHMO14 :  salaire net moyen par heure pour le travailleur
* SNHMF14 : salaire net moyen pour les femmes
* SNHMFC14 : salaire net moyen par heure pour les cadres féminins
* SNHMFP14 : salaire net moyen par heure pour les cadres moyens féminins
* SNHMFE14 : salaire net moyen par heure pour une employée 
* SNHMFO14 : salaire net moyen par heure pour une travailleuse 
* SNHMH14 : salaire net moyen pour un homme
* SNHMHC14 : salaire net moyen par heure pour un cadre masculin
* SNHMHP14 : salaire net moyen par heure pour les cadres moyens masculins
* SNHMHE14 : salaire net moyen par heure pour un employé masculin
* SNHMHO14 : salaire net moyen par heure pour un travailleur masculin
* SNHM1814 : salaire net moyen par heure pour les 18-25 ans
* SNHM2614 : salaire net moyen par heure pour les 26-50 ans
* SNHM5014 : salaire net moyen par heure pour les >50 ans
* SNHMF1814 : salaire net moyen par heure pour les femmes âgées de 18 à 25 ans
* SNHMF2614 : salaire net moyen par heure pour les femmes âgées de 26 à 50 ans
* SNHMF5014 : salaire net moyen par heure pour les femmes de plus de 50 ans
* SNHMH1814 : salaire net moyen par heure pour les hommes âgés de 18 à 25 ans
* SNHMH2614 : salaire net moyen par heure pour les hommes âgés de 26 à 50 ans
* SNHMH5014 : salaire net moyen par heure pour les hommes de plus de 50 ans


In [25]:
# Lecture du fichier csv
df3 = pd.read_csv('../0000-Initial/net_salary_per_town_categories.csv')
# Affichage des premières de la DF
df3.head()

,CODGEO,LIBGEO,SNHM14,SNHMC14,SNHMP14,SNHME14,SNHMO14,SNHMF14,SNHMFC14,SNHMFP14,...,SNHMHO14,SNHM1814,SNHM2614,SNHM5014,SNHMF1814,SNHMF2614,SNHMF5014,SNHMH1814,SNHMH2614,SNHMH5014
0,01004,Ambérieu-en-Bugey,13.7,24.2,15.5,10.3,11.2,11.6,19.1,13.2,...,11.6,10.5,13.7,16.1,9.7,11.8,12.5,11.0,14.9,18.6
1,01007,Ambronay,13.5,22.1,14.7,10.7,11.4,11.9,19.0,13.3,...,11.7,9.8,13.8,14.6,9.2,12.2,12.5,10.2,14.9,16.4
2,01014,Arbent,13.5,27.6,15.6,11.1,11.1,10.9,19.5,11.7,...,11.8,9.3,13.3,16.0,8.9,10.6,12.5,9.6,15.1,18.6
3,01024,Attignat,12.9,21.8,14.1,11.0,11.3,11.4,19.0,13.0,...,11.6,9.6,12.9,14.2,9.3,11.4,12.2,9.7,13.8,15.9
4,01025,Bâgé-la-Ville,13.0,22.8,14.1,10.5,11.1,11.6,19.4,13.6,...,11.4,9.4,12.8,15.2,9.0,11.8,12.3,9.7,13.4,16.9


In [26]:
# CODGEO : ID géographique de la ville
# LIBGEO : nom de la ville
# SNHM14 : salaire net moyen
# SNHMC14 : salaire net moyen par heure pour les cadres
# SNHMP14 : salaire net moyen par heure pour un cadre moyen
# SNHME14 : salaire net moyen par heure pour l'employé
# SNHMO14 : salaire net moyen par heure pour le travailleur
# SNHMF14 : salaire net moyen pour les femmes
# SNHMFC14 : salaire net moyen par heure pour les cadres féminins
# SNHMFP14 : salaire net moyen par heure pour les cadres moyens féminins
# SNHMFE14 : salaire net moyen par heure pour une employée
# SNHMFO14 : salaire net moyen par heure pour une travailleuse
# SNHMH14 : salaire net moyen pour un homme
# SNHMHC14 : salaire net moyen par heure pour un cadre masculin
# SNHMHP14 : salaire net moyen par heure pour les cadres moyens masculins
# SNHMHE14 : salaire net moyen par heure pour un employé masculin
# SNHMHO14 : salaire net moyen par heure pour un travailleur masculin
# SNHM1814 : salaire net moyen par heure pour les 18-25 ans
# SNHM2614 : salaire net moyen par heure pour les 26-50 ans
# SNHM5014 : salaire net moyen par heure pour les >50 ans
# SNHMF1814 : salaire net moyen par heure pour les femmes âgées de 18 à 25 ans
# SNHMF2614 : salaire net moyen par heure pour les femmes âgées de 26 à 50 ans
# SNHMF5014 : salaire net moyen par heure pour les femmes de plus de 50 ans
# SNHMH1814 : salaire net moyen par heure pour les hommes âgés de 18 à 25 ans
# SNHMH2614 : salaire net moyen par heure pour les hommes âgés de 26 à 50 ans
# SNHMH5014 : salaire net moyen par heure pour les hommes de plus de 50 ans
df3_SNHM14 = df3.drop(labels='LIBGEO',axis='columns')
df3_SNHM14

,CODGEO,SNHM14,SNHMC14,SNHMP14,SNHME14,SNHMO14,SNHMF14,SNHMFC14,SNHMFP14,SNHMFE14,...,SNHMHO14,SNHM1814,SNHM2614,SNHM5014,SNHMF1814,SNHMF2614,SNHMF5014,SNHMH1814,SNHMH2614,SNHMH5014
0,01004,13.7,24.2,15.5,10.3,11.2,11.6,19.1,13.2,10.1,...,11.6,10.5,13.7,16.1,9.7,11.8,12.5,11.0,14.9,18.6
1,01007,13.5,22.1,14.7,10.7,11.4,11.9,19.0,13.3,10.6,...,11.7,9.8,13.8,14.6,9.2,12.2,12.5,10.2,14.9,16.4
2,01014,13.5,27.6,15.6,11.1,11.1,10.9,19.5,11.7,10.8,...,11.8,9.3,13.3,16.0,8.9,10.6,12.5,9.6,15.1,18.6
3,01024,12.9,21.8,14.1,11.0,11.3,11.4,19.0,13.0,10.3,...,11.6,9.6,12.9,14.2,9.3,11.4,12.2,9.7,13.8,15.9
4,01025,13.0,22.8,14.1,10.5,11.1,11.6,19.4,13.6,10.2,...,11.4,9.4,12.8,15.2,9.0,11.8,12.3,9.7,13.4,16.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,97420,12.9,24.5,15.4,10.9,10.9,12.4,21.6,14.7,10.8,...,11.1,9.2,12.8,15.4,9.3,12.3,14.7,9.2,13.1,15.8
5132,97421,10.4,17.3,13.8,9.6,9.8,9.8,13.2,11.4,9.6,...,10.0,9.0,10.6,11.4,8.9,9.9,10.5,9.0,11.0,11.7
5133,97422,12.0,23.3,14.7,10.3,10.5,11.5,21.3,14.1,10.2,...,10.6,8.9,11.9,14.9,8.9,11.4,13.9,9.0,12.1,15.4
5134,97423,11.4,22.6,13.4,10.1,10.5,10.8,15.9,12.4,10.1,...,10.5,8.9,11.5,13.1,8.9,11.1,11.4,9.0,11.8,13.7


In [27]:
# Fusion de all content pop entreprises avec les données de salaire
all_content_2014 = pd.merge(all_content_pop_entr,df3_SNHM14,how='left',on='CODGEO')
all_content_2014.shape

(36681, 46)

In [28]:
# Sauvegarde de la DF all_content_pop dans un fichier csv
all_content_2014.to_csv("all_content_2014_v4.csv",encoding= "utf-8")